# Equational Numbers

I recently wondered about numbers that could be converted into valid equations by inserting operations and an equals sign. Some examples:
- 123 (1 + 2 = 3)
- 725 (7 = 2 + 5)
- 1010250 (10 x 10 / 2 = 50)

Of course, the first step was to search OEIS. I found this closely related but slightly different sequence : https://oeis.org/A068520 . It's basically the same idea, but the equals sign is constrained to be in the final position, whereas in my thought it could be anywhere.

I was a wondering about how to code this up, and was quite muddled, but looking at the code from that OEIS link (https://github.com/archmageirvine/joeis/blob/master/src/irvine/oeis/a068/A068520.java), I can see that doing a top down construction of the operation tree should work well.

So, let me explore this further.

Beyond the primary question of what are the numbers that meet this constraint, many questions are possible:
- what is the smallest N digit equation number?
- what is the first equation number to use all 4 different operators ?
- what is the first equation number to use all 10 digits ?
- Both of those combined?
- First equation number with N distinct valid equations?
- What happens if you also allow exponentation as an operator ?
- How does the fraction of equational numbers evolve as the numbers grow larger ?

Let's go.

In [39]:
from enum import Flag, auto
# EI = Equation number Info
class EI(Flag):
    NOTHING = 0
    MATCH_LIST = auto()
    MATCH_CNT = auto()
    MATCH_PCT = auto()    
    MATCH_EQNS = auto()
    FIRSTS = auto()
    NON_MATCH_LIST = auto()
    NON_MATCH_CNT = auto()    
    NON_MATCH_PCT = auto()       
    

set_digits = set([str(i) for i in range(10)])
set_operators = set(['+', '-', '*', '/', '**'])
set_operators_alt = set(['+', '-', '*', '/', '^'])
set_brackets = set(['(', ')'])

def all_equations(n, equals_anywhere=True, allow_exp=False):
    str_n = str(n)
    len_n = len(str_n)
    split_pos_start = len_n - 1 if not equals_anywhere else 2 if (len_n > 1 and str_n[0] == '-') else 1
    split_pos_end = len_n - 1    
    for split_pos in range(split_pos_start, split_pos_end + 1):
        left = str_n[:split_pos]
        right = str_n[split_pos:]
        left_exprs = all_expressions(left, allow_exp)
        right_exprs = all_expressions(right, allow_exp)
        for left_expr in left_exprs:
            for right_expr in right_exprs:
                if left_expr and right_expr:                    
                    yield f"{left_expr}=={right_expr}"

from functools import lru_cache

@lru_cache(maxsize=8192)
def all_expressions(str_n, allow_exp=False):
    expressions = [str_n]
    len_n = len(str_n)
    split_pos_start = 2 if (len_n > 1 and str_n[0] == '-') else 1
    split_pos_end = len_n - 1
    active_operators = [op for op in set_operators if op != '**' or allow_exp]
    for split_pos in range(split_pos_start, split_pos_end + 1):
        left = str_n[:split_pos]
        right = str_n[split_pos:]
        left_exprs = all_expressions(left, allow_exp)
        right_exprs = all_expressions(right, allow_exp)
        for left_expr in left_exprs:
            for right_expr in right_exprs:
                if left_expr and right_expr:
                    for op in active_operators:
                        expressions.append(f"({left_expr}){op}({right_expr})")
                    
    return expressions

# Note that some equations like '22=022' will return False because python does not allow leading zeros.
# But such numbers will still have valid equations like 22=0+22.
# So, which numbers do and don't have valid equations stay the same.
# But equation counts may differ from other implementations.
# It's kinda fine because humans usually don't think of numbers with leading zeroes.
# And it is definitely better than interpreting '022' as an octal number, i.e. decimal 18.
def is_true_equation(equation):
    try:
        ee = eval(equation)
        return ee is True
    except (SyntaxError, ZeroDivisionError, NameError):
        return False

def equation_signature(equation):
    equation_mod = equation
    equation_mod = equation_mod.replace('**', '^')
    set_e = set(equation)
    set_d = set_e & set_digits
    set_o = set_e & set_operators_alt
    #set_b = set_e & set_brackets
    return f"num_digits={len(set_d)}, num_ops={len(set_o)}"

from pprint import pprint
def find_equational_numbers(min_n=1, max_n=1000, 
                            equals_anywhere=True,
                            allow_exp=False,
                            print_info : EI = EI.MATCH_CNT|EI.MATCH_LIST,
                            return_info : EI = EI.NOTHING):
    desired_info = print_info | return_info if return_info else print_info
    equational_numbers= []
    non_equational_numbers = []
    equational_numbers_with_equations = {}
    signatures_seen = []
    counts_seen = []
    for n in range(min_n, max_n + 1):        
        true_equations = []                
        for equation in all_equations(n, equals_anywhere, allow_exp):
            if is_true_equation(equation):                
                true_equations.append(equation)
                if (EI.FIRSTS in desired_info):
                    sig = equation_signature(equation)
                    if sig not in signatures_seen:
                        print(f"First number with {sig}: {n}: {equation}")
                        signatures_seen.append(sig)                
                if (not(EI.MATCH_EQNS in desired_info or EI.FIRSTS in desired_info)):
                    break
        if true_equations:
            if (EI.MATCH_EQNS in desired_info):
               equational_numbers_with_equations[n] = true_equations
            if (EI.MATCH_LIST in desired_info or EI.MATCH_CNT in desired_info or EI.MATCH_PCT in desired_info):
                equational_numbers.append(n)
            if (EI.FIRSTS in desired_info):
                num_equations = len(true_equations)
                if num_equations not in counts_seen:
                    print(f"First number with {num_equations} distinct equations: {n}:", end=' ')
                    pprint(true_equations, compact=True, width=120)
                    counts_seen.append(num_equations)                
        else:
            if (EI.NON_MATCH_LIST in desired_info or EI.NON_MATCH_CNT in desired_info or EI.NON_MATCH_PCT in desired_info):
                non_equational_numbers.append(n)
    equational_count = len(equational_numbers)
    non_equational_count = len(non_equational_numbers)
    equational_percentage = 100*equational_count /(max_n - min_n + 1)
    non_equational_percentage = 100*non_equational_count /(max_n - min_n + 1)
    if (EI.MATCH_CNT in print_info or EI.MATCH_PCT in print_info):
        print(f"From {min_n} to {max_n}: {equational_count} equational numbers found ({equational_percentage:.2f}%)")
    if (EI.MATCH_LIST in print_info):
        pprint(equational_numbers, width=120, compact=True)
    if (EI.NON_MATCH_CNT in print_info or EI.NON_MATCH_PCT in print_info):
        print(f"From {min_n} to {max_n}: {non_equational_count} non-equational numbers found ({non_equational_percentage:.2f}%)")
    if (EI.NON_MATCH_LIST in print_info):
        pprint(non_equational_numbers, width=120, compact=True)
    if (EI.MATCH_EQNS in print_info):
        print(f"From {min_n} to {max_n}: {len(equational_numbers_with_equations)} numbers with equations found")
        pprint(equational_numbers_with_equations, width=120)
    match return_info:
        case EI.MATCH_CNT:
            return equational_count
        case EI.MATCH_PCT:
            return equational_percentage
        case EI.MATCH_LIST:
            return equational_numbers
        case EI.NON_MATCH_CNT:
            return non_equational_count
        case EI.NON_MATCH_PCT:
            return non_equational_percentage
        case EI.NON_MATCH_LIST:
            return non_equational_numbers
        case EI.MATCH_EQNS:
            return equational_numbers_with_equations
        case _:
            return None

# Equals sign in final position - A068520

https://oeis.org/A068520  https://oeis.org/A068520/b068520.txt .  Recreating this known sequence helps make sure the code works as expected.

In [40]:
find_equational_numbers(max_n=1000, equals_anywhere=False, allow_exp=False)

From 1 to 1000: 144 equational numbers found (14.40%)
[11, 22, 33, 44, 55, 66, 77, 88, 99, 100, 101, 110, 111, 112, 122, 123, 133, 134, 144, 145, 155, 156, 166, 167, 177,
 178, 188, 189, 199, 200, 202, 211, 212, 213, 220, 221, 224, 235, 236, 246, 248, 257, 268, 279, 300, 303, 312, 313, 314,
 321, 325, 326, 330, 331, 336, 339, 347, 358, 369, 400, 404, 413, 414, 415, 422, 426, 428, 431, 437, 440, 441, 448, 459,
 500, 505, 514, 515, 516, 523, 527, 532, 538, 541, 549, 550, 551, 600, 606, 615, 616, 617, 623, 624, 628, 632, 633, 639,
 642, 651, 660, 661, 700, 707, 716, 717, 718, 725, 729, 734, 743, 752, 761, 770, 771, 800, 808, 817, 818, 819, 824, 826,
 835, 842, 844, 853, 862, 871, 880, 881, 900, 909, 918, 919, 927, 933, 936, 945, 954, 963, 972, 981, 990, 991, 1000]


In [41]:
# To demonstrate printouts
# find_equational_numbers(max_n=331, 
#                        equals_in_last_place_only=True, allow_exponentiation=False,
#                        print_info=EI.MATCH_CNT|EI.MATCH_LIST|EI.MATCH_EQNS|EI.NON_MATCH_CNT|EI.NON_MATCH_LIST|EI.FIRSTS)

# With Equals sign in any position.

In [42]:
find_equational_numbers(max_n=1000, equals_anywhere=True, allow_exp=False)

From 1 to 1000: 185 equational numbers found (18.50%)
[11, 22, 33, 44, 55, 66, 77, 88, 99, 100, 101, 110, 111, 112, 121, 122, 123, 132, 133, 134, 143, 144, 145, 154, 155,
 156, 165, 166, 167, 176, 177, 178, 187, 188, 189, 198, 199, 200, 202, 211, 212, 213, 220, 221, 224, 231, 235, 236, 242,
 246, 248, 253, 257, 263, 264, 268, 275, 279, 284, 286, 297, 300, 303, 312, 313, 314, 321, 325, 326, 330, 331, 336, 339,
 341, 347, 352, 358, 362, 363, 369, 374, 385, 393, 396, 400, 404, 413, 414, 415, 422, 426, 428, 431, 437, 440, 441, 448,
 451, 459, 462, 473, 482, 484, 495, 500, 505, 514, 515, 516, 523, 527, 532, 538, 541, 549, 550, 551, 561, 572, 583, 594,
 600, 606, 615, 616, 617, 623, 624, 628, 632, 633, 639, 642, 651, 660, 661, 671, 682, 693, 700, 707, 716, 717, 718, 725,
 729, 734, 743, 752, 761, 770, 771, 781, 792, 800, 808, 817, 818, 819, 824, 826, 835, 842, 844, 853, 862, 871, 880, 881,
 891, 900, 909, 918, 919, 927, 933, 936, 945, 954, 963, 972, 981, 990, 991, 1000]


This has a lot of commonality with https://oeis.org/A338214, but that is restricted to just multiply and add.

# With Exponentiation as well.

In [43]:
find_equational_numbers(max_n=1000, equals_anywhere=False, allow_exp=True)

From 1 to 1000: 162 equational numbers found (16.20%)
[11, 22, 33, 44, 55, 66, 77, 88, 99, 100, 101, 110, 111, 112, 121, 122, 123, 131, 133, 134, 141, 144, 145, 151, 155,
 156, 161, 166, 167, 171, 177, 178, 181, 188, 189, 191, 199, 200, 201, 202, 211, 212, 213, 220, 221, 224, 235, 236, 238,
 246, 248, 257, 268, 279, 300, 301, 303, 312, 313, 314, 321, 325, 326, 329, 330, 331, 336, 339, 347, 358, 369, 400, 401,
 404, 413, 414, 415, 422, 426, 428, 431, 437, 440, 441, 448, 459, 500, 501, 505, 514, 515, 516, 523, 527, 532, 538, 541,
 549, 550, 551, 600, 601, 606, 615, 616, 617, 623, 624, 628, 632, 633, 639, 642, 651, 660, 661, 700, 701, 707, 716, 717,
 718, 725, 729, 734, 743, 752, 761, 770, 771, 800, 801, 808, 817, 818, 819, 824, 826, 835, 842, 844, 853, 862, 871, 880,
 881, 900, 901, 909, 918, 919, 927, 933, 936, 945, 954, 963, 972, 981, 990, 991, 1000]


In [44]:
find_equational_numbers(max_n=1000, equals_anywhere=True, allow_exp=True)

From 1 to 1000: 219 equational numbers found (21.90%)
[11, 22, 33, 44, 55, 66, 77, 88, 99, 100, 101, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
 130, 131, 132, 133, 134, 140, 141, 143, 144, 145, 150, 151, 154, 155, 156, 160, 161, 165, 166, 167, 170, 171, 176, 177,
 178, 180, 181, 187, 188, 189, 190, 191, 198, 199, 200, 201, 202, 211, 212, 213, 220, 221, 224, 231, 235, 236, 238, 242,
 246, 248, 253, 257, 263, 264, 268, 275, 279, 284, 286, 297, 300, 301, 303, 312, 313, 314, 321, 325, 326, 329, 330, 331,
 336, 339, 341, 347, 352, 358, 362, 363, 369, 374, 385, 393, 396, 400, 401, 404, 413, 414, 415, 422, 426, 428, 431, 437,
 440, 441, 448, 451, 459, 462, 473, 482, 484, 495, 500, 501, 505, 514, 515, 516, 523, 527, 532, 538, 541, 549, 550, 551,
 561, 572, 583, 594, 600, 601, 606, 615, 616, 617, 623, 624, 628, 632, 633, 639, 642, 651, 660, 661, 671, 682, 693, 700,
 701, 707, 716, 717, 718, 725, 729, 734, 743, 752, 761, 770, 771, 781, 792, 800, 801, 808, 817, 818, 81

Unsurprisingly, relaxing constraints allows more numbers to work.
These 2 do not match anything on OEIS.
Let's see the differences.

In [45]:
A = set(find_equational_numbers(max_n=222, equals_anywhere=False, allow_exp=False, print_info=EI.NOTHING, return_info=EI.MATCH_LIST))
B = set(find_equational_numbers(max_n=222, equals_anywhere=True, allow_exp=False, print_info=EI.NOTHING, return_info=EI.MATCH_LIST))
C = set(find_equational_numbers(max_n=222, equals_anywhere=False, allow_exp=True, print_info=EI.NOTHING, return_info=EI.MATCH_LIST))
D = set(find_equational_numbers(max_n=222, equals_anywhere=True, allow_exp=True, print_info=EI.NOTHING, return_info=EI.MATCH_LIST))
print("Numbers newly allowed by free equals")
pprint(sorted(list(B-A)), width=120)
print("Numbers newly allowed by exponents")
pprint(sorted(list(C-A)), width=120)
print("Numbers newly allowed by free equals (with exponents)")
pprint(sorted(list(D-C)), width=120)
print("Numbers newly allowed by exponents (with free equals)")
pprint(sorted(list(D-B)), width=120)


Numbers newly allowed by free equals
[121, 132, 143, 154, 165, 176, 187, 198]
Numbers newly allowed by exponents
[121, 131, 141, 151, 161, 171, 181, 191, 201]
Numbers newly allowed by free equals (with exponents)
[113, 114, 115, 116, 117, 118, 119, 120, 130, 132, 140, 143, 150, 154, 160, 165, 170, 176, 180, 187, 190, 198]
Numbers newly allowed by exponents (with free equals)
[113, 114, 115, 116, 117, 118, 119, 120, 130, 131, 140, 141, 150, 151, 160, 161, 170, 171, 180, 181, 190, 191, 201]


# Distribution

Since the number of ways to construct equations grows exponentially with the number of digits, I wonder if all sufficiently larger numbers are "equational numbers". Let's check.

In [49]:
for d in range(1,6):
    min_n = 10**(d-1)
    max_n = 10**d - 1
    find_equational_numbers(min_n=min_n, max_n=max_n,equals_anywhere=True, allow_exp=False, print_info=EI.NON_MATCH_CNT)

From 1 to 9: 9 non-equational numbers found (100.00%)
From 10 to 99: 81 non-equational numbers found (90.00%)
From 100 to 999: 725 non-equational numbers found (80.56%)
From 1000 to 9999: 4718 non-equational numbers found (52.42%)
From 10000 to 99999: 11890 non-equational numbers found (13.21%)


Saved results:<pre>
From 1 to 9: 9 non-equational numbers found (100.00%)
From 10 to 99: 81 non-equational numbers found (90.00%)
From 100 to 999: 725 non-equational numbers found (80.56%)
From 1000 to 9999: 4718 non-equational numbers found (52.42%)
From 10000 to 99999: 11890 non-equational numbers found (13.21%)</pre>

In [50]:
find_equational_numbers(min_n=1000, max_n=9999,equals_anywhere=True, allow_exp=False, print_info=EI.NON_MATCH_CNT)

From 1000 to 9999: 4718 non-equational numbers found (52.42%)


From 5000 to 6000: 554 non-equational numbers found (55.34%)

In [51]:
all_expressions.cache_info()

CacheInfo(hits=889629, misses=89173, maxsize=8192, currsize=8192)